In [1]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.vectorstores import Chroma
from datetime import datetime
from langchain.schema import Document
from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings,)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.chains import ConversationalRetrievalChain

C:\Users\Maria Waheed\anaconda3\envs\llm_2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [2]:
vectorstore_load=Chroma(persist_directory="./chromadb_data_v1", embedding_function=embedding_function)

In [3]:
len(vectorstore_load.get()['documents'])

205

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

llm = ChatOpenAI(temperature=0.7, model=llm_model, api_key=openai.api_key)

C:\Users\Maria Waheed\anaconda3\envs\llm_2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
# Get user input
content_schema = ResponseSchema(name="output",description=""" The output should be in a JSON/dictionary format the keys should be {"date":"", "time":"", "info":""} """)
response_schemas = [content_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

user_input = input("How can I help: ")

chat = ChatOpenAI(temperature=0.0, model=llm_model,api_key=openai.api_key)
prompt_template = ChatPromptTemplate.from_template(
   """ Given the user input find the date and time if not provided add null; convert it in time format remove PM or AM if needed and if a time range is given pick the first time also extract the insight if any is provided:
   Following is the given content{content}
   
    ~~Only generate the dictionary and nothing else~~.
    {format_instructions}
    """
)

llm_messages = prompt_template.format_messages(content=user_input, format_instructions=format_instructions)
response = chat(llm_messages)

print(response.content)
output_dict = output_parser.parse(response.content)

How can I help:  what happened around 12:30


C:\Users\Maria Waheed\anaconda3\envs\llm_2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


```json
{
	"output": {
		"date": null,
		"time": "12:30",
		"info": "what happened around"
	}
}
```


In [7]:
time_str=output_dict["output"]["time"]
from datetime import datetime, timedelta
try:
        time_obj = datetime.strptime(time_str, "%H:%M:%S")
except ValueError:
    try:
        time_obj = datetime.strptime(time_str, "%H:%M")
    except ValueError:
        try:
            time_obj = datetime.strptime(time_str, "%H")
        except ValueError:
            raise ValueError("Invalid time format. Please provide time in the format 'HH', 'HH:MM', or 'HH:MM:SS'.")

start_time = time_obj - timedelta(minutes=10)
end_time = time_obj + timedelta(minutes=10)

# Format the times back to strings in 24-hour format
start_time_str = start_time.strftime("%H:%M")
end_time_str = end_time.strftime("%H:%M")

print(f"Start time: {start_time_str}, End time: {end_time_str}") 



date_hardwork="2024-06-20"
def getIntFromDatetime(datetime_obj):
    return int(datetime_obj.strftime('%Y%m%d'))

Start time: 12:20, End time: 12:40


In [8]:
"""
K=10
end_time_loop = end_time + timedelta(minutes=1)
counter= start_time
list_documents_retrieved=[]

while True:
    if counter< end_time_loop:
        chroma_retriever = VectorStoreRetriever(vectorstore=vectorstore_load, metadata={"date":output_dict["output"]["date"],"time":counter ,"k":K},)
        retrieved_docs = chroma_retriever.get_relevant_documents(user_input)
        print(len(retrieved_docs))
        for doc in retrieved_docs:
            if doc not in list_documents_retrieved:
                list_documents_retrieved.append(doc)
    else:
        break
    counter = counter + timedelta(minutes=1)
print(f"Total documents retrieved: {len(list_documents_retrieved)}")
"""

'\nK=10\nend_time_loop = end_time + timedelta(minutes=1)\ncounter= start_time\nlist_documents_retrieved=[]\n\nwhile True:\n    if counter< end_time_loop:\n        chroma_retriever = VectorStoreRetriever(vectorstore=vectorstore_load, metadata={"date":output_dict["output"]["date"],"time":counter ,"k":K},)\n        retrieved_docs = chroma_retriever.get_relevant_documents(user_input)\n        print(len(retrieved_docs))\n        for doc in retrieved_docs:\n            if doc not in list_documents_retrieved:\n                list_documents_retrieved.append(doc)\n    else:\n        break\n    counter = counter + timedelta(minutes=1)\nprint(f"Total documents retrieved: {len(list_documents_retrieved)}")\n'

In [9]:
str_formatted_time = datetime.strptime(start_time_str, "%H:%M").strftime("%H:%M:%S")
print("Formatted time:", str_formatted_time)

end_formatted_time = datetime.strptime(end_time_str, "%H:%M").strftime("%H:%M:%S")
print("Formatted time:", end_formatted_time)

# Combine the date and time
str_date_time_str = f"{date_hardwork} {str_formatted_time}"
end_date_time_str = f"{date_hardwork} {end_formatted_time}"
# Create the datetime object
start_date_time_obj = datetime.strptime(str_date_time_str, "%Y-%m-%d %H:%M:%S")
end_date_time_obj = datetime.strptime(end_date_time_str, "%Y-%m-%d %H:%M:%S")
print(start_date_time_obj,"<<--->>",end_date_time_obj)

Formatted time: 12:20:00
Formatted time: 12:40:00
2024-06-20 12:20:00 <<--->> 2024-06-20 12:40:00


In [10]:
timestamp_s = start_date_time_obj.timestamp()
timestamp_e = end_date_time_obj.timestamp()
print("Timestamp_start:", timestamp_s)
print("Timestamp_end:", timestamp_e)

Timestamp_start: 1718868000.0
Timestamp_end: 1718869200.0


In [16]:
type(timestamp_s)

float

In [11]:
filter_query = {'$and': [{'time_stamp': {'$gte': timestamp_s}}, {'time_stamp': {'$lte': timestamp_e}}]}

In [12]:
filter_query

{'$and': [{'time_stamp': {'$gte': 1718868000.0}},
  {'time_stamp': {'$lte': 1718869200.0}}]}

In [13]:
K=5
chroma_retriever = VectorStoreRetriever(vectorstore=vectorstore_load, search_kwargs={"filter":filter_query,"k":K},)
retrieved_docs = chroma_retriever.get_relevant_documents(output_dict["output"]["info"])
print(len(retrieved_docs))

0


C:\Users\Maria Waheed\anaconda3\envs\llm_2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
filtered_results = vectorstore_load.similarity_search(user_input)

In [18]:
type(filtered_results[0].metadata["time_stamp"])

float

In [19]:
filtered_results[0].metadata

{'date': '2024-06-20',
 'location': 'Imtiaz Supermarket',
 'path': "D:\\core9_work\\Amir's Land rover AI Enablement\\dummy_chromadb\\frame_19.jpg",
 'time': '12:05:20',
 'time_stamp': 1718867120.0}

In [152]:
chosen=""
for i in range(len(retrieved_docs)):
    if str(filtered_results[0].metadata["path"])==str(retrieved_docs[i].metadata["path"]):
        chosen=filtered_results[0].metadata["path"]

In [153]:
chosen

"D:\\core9_work\\Amir's Land rover AI Enablement\\dummy_chromadb\\frame_26.jpg"

In [90]:
llm_model = "gpt-4o"
chat = ChatOpenAI(temperature=0.7, model=llm_model,api_key=openai.api_key)

In [22]:
#!pip install langchain

In [24]:
from langchain_core.messages import HumanMessage
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import base64

model = ChatOpenAI(model="gpt-4o")

In [25]:
image_path="D:\\core9_work\\Amir's Land rover AI Enablement\\dummy_chromadb\\frame_26.jpg"
with open(image_path,"rb") as image_file:
    image_data=base64.b64encode(image_file.read()).decode("utf=8")

In [29]:
content_prompt="Describe what is going in the image"
message=HumanMessage(
    content=[
        {"type":"text","text":content_prompt},
        {
            "type":"image_url",
            "image_url":{"url":f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response= model.invoke([message])
print(response.content)

The image depicts a scene of a multi-lane highway with light traffic. There are a few vehicles visible on the road, including a car in the middle lane and a truck further ahead. The highway is flanked by trees on both sides, and the weather appears to be clear with good visibility. The road is divided by white lane markings, and there are streetlights along the median. An overhead structure, possibly an overpass or sign frame, is visible at the top of the image. The overall impression is of a calm and orderly drive on a well-maintained highway.
